In [ ]:
import pandas as pd
import numpy as np
import cv2
from params import args
import os

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

%reload_ext autoreload
%autoreload 2

In [ ]:
train = pd.read_csv('data/train_proc_v2.csv')

In [ ]:
test = pd.read_csv('data/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
depth = pd.read_csv('data/depths.csv')

In [ ]:
test = test.merge(depth)

In [ ]:
test[test.z==63].head()

In [ ]:
path = os.path.join(args.masks_dir,'{}.png'.format('88839f49f9'))
m = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

In [ ]:
train.head()

In [ ]:
np.max(m)

In [ ]:
args.augmentation_name

In [ ]:
from params import args
from augmentations import get_augmentations
def read_image(path, input_size):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # img = cv2.resize(img, input_size)
    # [:,:,::-1]

    return img
    
def read_mask(path, input_size):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # mask = cv2.resize(mask, input_size)

    return mask  

id = '0f5ada4dc3'

# Read with resizing
img = read_image(os.path.join(args.images_dir,'{}.png'.format(id)), None)
mask = read_mask(os.path.join(args.masks_dir,'{}.png'.format(id)), None)

plt.figure(figsize=(3,3))
plt.imshow(img)

print(img[0][0])

# Random Crops during training
augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))

if augs:
    data = {"image": img, "mask": mask}
    augmented = augs(**data)
    img, mask = augmented["image"], augmented["mask"]
    
plt.figure(figsize=(3,3))
plt.imshow(img)
print(img[0][0])

In [ ]:
train.sort_values('rle_mask')

In [ ]:
borders = []
all_imgs_test = []

for id in test[test.z==942].id.values:
    path = os.path.join(args.test_folder,'{}.png'.format(id))
    img = cv2.imread(path, cv2.IMREAD_COLOR)    
    all_imgs_test.append(img)

In [ ]:
borders = []
all_imgs_train = []

for id in train[train.z==942].id.values:
    path = os.path.join(args.images_dir,'{}.png'.format(id))
    path_mask = os.path.join(args.masks_dir,'{}.png'.format(id))
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)     
    all_imgs_train.append(img)

In [ ]:
plt.imshow(np.hstack([all_imgs_test[-1],all_imgs_train[-1]]))

In [ ]:
plt.imshow(np.hstack(all_imgs_test))

In [ ]:
plt.imshow(np.hstack(all_imgs_train))

In [ ]:
len()

In [ ]:
borders = []

for id in train[train.z==63].id.values:
    path = os.path.join(args.images_dir,'{}.png'.format(id))
    path_mask = os.path.join(args.masks_dir,'{}.png'.format(id))
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)     
    
    plt.figure(figsize=(4,4))
    plt.imshow(img)

In [ ]:
train.sort_values('z')

In [ ]:
borders = []

for id in train.id.values:
    path = os.path.join(args.images_dir,'{}.png'.format(id))
    path_mask = os.path.join(args.masks_dir,'{}.png'.format(id))
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)     
    
    up_border = img[0,:]
    down_border = img[-1,:]
    left_border = img[:,0]
    right_border = img[:,-1]
    
    borders.extend(up_border,down_border,left_border,right_border)

# prepare df with all data
lens = [len(border) for border in borders]
img_idx = list(range(len(border)//4))*4
position = ['up','down','left','right']*len(len(border)//4)
nn = [None]*len(position)

In [ ]:
img[0,:].shape

In [ ]:
height, width = img.shape[:2]

In [ ]:
import tqdm

In [ ]:
def load_raw_data(image_size=(101, 101), space = 'bgr',load_mask=True):
    """Load raw data."""
    # Python lists to store the training images/masks and test images.
    x_train, y_train, x_test = [],[],[]

    for id in tqdm.tqdm_notebook(train.id.values):
        path = os.path.join(args.images_dir,'{}.png'.format(id))
        path_mask = os.path.join(args.masks_dir,'{}.png'.format(id))
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if load_mask:
            mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)
            mask = np.expand_dims(mask, axis=2)
            y_train.append(mask)
        x_train.append(img)
        
    # Read and resize test images. 
    
    for id in tqdm.tqdm_notebook(test.id.values):
        path = os.path.join(args.test_folder,'{}.png'.format(id))
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        x_test.append(img)

    # Transform lists into 4-dim numpy arrays.
    x_train = np.array(x_train)
    #if load_mask:
    y_train = np.array(y_train)
    #return x_train, y_train
    #y_train = np.expand_dims(np.array(y_train), axis=4)
    x_test = np.array(x_test)
    print('Data loaded')
    if load_mask:
        return x_train, y_train, x_test
    else:
        return x_train, x_test

In [ ]:
x_train, y_train, x_test = load_raw_data()

In [ ]:
x_train.shape

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def combine_images(data,indexes):
    """ Combines img from data using indexes as follows:
        0 1
        2 3 
    """
    up = np.hstack([data[indexes[0]],data[indexes[1]]])
    down = np.hstack([data[indexes[2]],data[indexes[3]]])
    full = np.vstack([up,down])
    return full

def make_mosaic(data,return_connectivity = False, plot_images = False,external_df = None):
    """Find images with simular borders and combine them to one big image"""
    if external_df is not None:
        external_df['mosaic_idx'] = np.nan
        external_df['mosaic_position'] = np.nan
        # print(external_df.head())
    
    # extract borders from images
    borders = []
    for x in data:
        borders.extend([x[0,:,:].flatten(),x[-1,:,:].flatten(),
                        x[:,0,:].flatten(),x[:,-1,:].flatten()])
    borders = list(np.array(borders))

    # prepare df with all data
    lens = [len(border) for border in borders]
    img_idx = list(range(len(data)))*4
    img_idx.sort()
    position = ['up','down','left','right']*len(data)
    nn = [None]*len(position)
    #print(np.vstack([img_idx,position,borders,lens,nn]).shape)
    # df = pd.DataFrame([img_idx,position,borders,lens,nn])
    df = pd.DataFrame({'img_idx':img_idx,'position':position,'border':borders,'len':lens,'nn':nn})
#     df = pd.DataFrame(data=np.vstack([img_idx,position,borders,,nn]).T,
#                       columns=['img_idx','position','border',,'nn'])
    uniq_lens = df['len'].unique()
    
    for idx,l in enumerate(uniq_lens):
        # fit NN on borders of certain size with 1 neighbor
        nn = NearestNeighbors(n_neighbors=1).fit(np.stack(df[df.len == l]['border'].values))
        distances, neighbors = nn.kneighbors()
        real_neighbor = np.array([None]*len(neighbors))
        distances, neighbors = distances.flatten(),neighbors.flatten()

        # if many borders are close to one, we want to take only the closest
        uniq_neighbors = np.unique(neighbors)

        # difficult to understand but works :c
        for un_n in uniq_neighbors:
            # min distance for borders with same nn
            min_index = list(distances).index(distances[neighbors == un_n].min())
            # check that min is double-sided
            double_sided = distances[neighbors[min_index]] == distances[neighbors == un_n].min()
            if double_sided and distances[neighbors[min_index]] < 1000:
                real_neighbor[min_index] = neighbors[min_index]
                real_neighbor[neighbors[min_index]] = min_index
        indexes = df[df.len == l].index
        for idx2,r_n in enumerate(real_neighbor):
            if r_n is not None:
                df['nn'].iloc[indexes[idx2]] = indexes[r_n]
    
    # img connectivity graph. 
    img_connectivity = {}
    for img in df.img_idx.unique():
        slc = df[df['img_idx'] == img]
        img_nn = {}

        # get near images_id & position
        for nn_border,position in zip(slc[slc['nn'].notnull()]['nn'],
                                      slc[slc['nn'].notnull()]['position']):

            # filter obvious errors when we try to connect bottom of one image to bottom of another
            # my hypotesis is that images were simply cut, without rotation
            if position == df.iloc[nn_border]['position']:
                continue
            img_nn[position] = df.iloc[nn_border]['img_idx']
        img_connectivity[img] = img_nn

    imgs = []
    indexes = set()
    mosaic_idx = 0
    
    # errors in connectivity are filtered 
    good_img_connectivity = {}
    for k,v in img_connectivity.items():
        if v.get('down') is not None:
            if v.get('right') is not None:
                # need down right image
                # check if both right and down image are connected to the same image in the down right corner
                if (img_connectivity[v['right']].get('down') is not None) and img_connectivity[v['down']].get('right') is not None:
                    if img_connectivity[v['right']]['down'] == img_connectivity[v['down']]['right']:
                        v['down_right'] = img_connectivity[v['right']]['down']
                        temp_indexes = [k,v['right'],v['down'],v['down_right']]
                        if (len(np.unique(temp_indexes)) < 4) or (len(indexes.intersection(temp_indexes)) > 0):
                            continue
                        # надо тут фильтровать что они не одинаковые
                        good_img_connectivity[k] = temp_indexes
                        indexes.update(temp_indexes)
                        imgs.append(combine_images(data,temp_indexes))
                        if external_df is not None:
                            external_df['mosaic_idx'].iloc[temp_indexes] = mosaic_idx
                            external_df['mosaic_position'].iloc[temp_indexes] = ['up_left','up_right','down_left','down_right']
                            mosaic_idx += 1
                        continue
            if v.get('left') is not None:
                # need down left image
                if img_connectivity[v['left']].get('down') is not None and img_connectivity[v['down']].get('left') is not None:
                    if img_connectivity[v['left']]['down'] == img_connectivity[v['down']]['left']:
                        v['down_left'] = img_connectivity[v['left']]['down']
                        temp_indexes = [v['left'],k,v['down_left'],v['down']]
                        if (len(np.unique(temp_indexes)) < 4) or (len(indexes.intersection(temp_indexes)) > 0):
                            continue
                        good_img_connectivity[k] = temp_indexes
                        indexes.update(temp_indexes)
                        imgs.append(combine_images(data,temp_indexes))
                        
                        if external_df is not None:
                            external_df['mosaic_idx'].iloc[temp_indexes] = mosaic_idx
                            external_df['mosaic_position'].iloc[temp_indexes] = ['up_left','up_right','down_left','down_right']
                            
                            mosaic_idx += 1 
                        continue
        if v.get('up') is not None:
            if v.get('right') is not None:
                # need up right image
                if img_connectivity[v['right']].get('up') is not None and img_connectivity[v['up']].get('right') is not None:
                    if img_connectivity[v['right']]['up'] == img_connectivity[v['up']]['right']:
                        v['up_right'] = img_connectivity[v['right']]['up']
                        temp_indexes = [v['up'],v['up_right'],k,v['right']]
                        if (len(np.unique(temp_indexes)) < 4) or (len(indexes.intersection(temp_indexes)) > 0):
                            continue
                        good_img_connectivity[k] = temp_indexes
                        indexes.update(temp_indexes)
                        imgs.append(combine_images(data,temp_indexes))
                        
                        if external_df is not None:
                            external_df['mosaic_idx'].iloc[temp_indexes] = mosaic_idx
                            external_df['mosaic_position'].iloc[temp_indexes] = ['up_left','up_right','down_left','down_right']
                            
                            mosaic_idx += 1 
                        continue
            if v.get('left') is not None:
                # need up left image
                if img_connectivity[v['left']].get('up') is not None and img_connectivity[v['up']].get('left') is not None:
                    if img_connectivity[v['left']]['up'] == img_connectivity[v['up']]['left']:
                        v['up_left'] = img_connectivity[v['left']]['up']
                        temp_indexes = [v['up_left'],v['up'],v['left'],k]
                        if (len(np.unique(temp_indexes)) < 4) or (len(indexes.intersection(temp_indexes)) > 0):
                            continue
                        good_img_connectivity[k] = temp_indexes
                        indexes.update(temp_indexes)
                        imgs.append(combine_images(data,temp_indexes))
                        
                        if external_df is not None:
                            external_df['mosaic_idx'].iloc[temp_indexes] = mosaic_idx
                            external_df['mosaic_position'].iloc[temp_indexes] = ['up_left','up_right','down_left','down_right']
                            
                            mosaic_idx += 1 
                        continue

    # same images are present 4 times (one for every piece) so we need to filter them
    print('Images before filtering: {}'.format(np.shape(imgs)))
    
    # can use np. unique only on images of one size, flatten first, then select
    flattened = np.array([i.flatten() for i in imgs])
    uniq_lens = np.unique([i.shape for i in flattened])
    filtered_imgs = []
    for un_l in uniq_lens:
        filtered_imgs.extend(np.unique(np.array([i for i in imgs if i.flatten().shape == un_l]),axis=0))
        
    filtered_imgs = np.array(filtered_imgs)
    print('Images after filtering: {}'.format(np.shape(filtered_imgs)))
    
    if return_connectivity:
        print(good_img_connectivity)
    
    if plot_images:
        for i in filtered_imgs:
            plt.imshow(i)
            plt.show()
            
    # list of not combined images. return if you need
    not_combined = list(set(range(len(data))) - indexes)
    
    if external_df is not None:
        #un_mos_id = external_df[external_df.mosaic_idx.notnull()].mosaic_idx.unique()
        #mos_dict = {k:v for k,v in zip(un_mos_id,range(len(un_mos_id)))}
        #external_df.mosaic_idx = external_df.mosaic_idx.map(mos_dict)
        ## print(temp.mosaic_idx.shape[0])
        ## print(len(temp.mosaic_idx[temp.mosaic_idx.isnull()] ))
        ## print(len(list(range(temp.mosaic_idx.shape[0]-len(temp.mosaic_idx[temp.mosaic_idx.isnull()]),
        ##                     temp.mosaic_idx.shape[0]))))
        external_df.loc[external_df[external_df['mosaic_idx'].isnull()].index,'mosaic_idx'] = range(
            int(np.nanmax(external_df.mosaic_idx.unique())) + 1,
            int(np.nanmax(external_df.mosaic_idx.unique())) + 1 + len(external_df.mosaic_idx[external_df.mosaic_idx.isnull()]))
        external_df['mosaic_idx'] = external_df['mosaic_idx'].astype(np.int32)
        if return_connectivity:
            return filtered_imgs, external_df, good_img_connectivity
        else:
            return filtered_imgs, external_df
    if return_connectivity:
        return filtered_imgs,good_img_connectivity
    else:
        return filtered_imgs

In [ ]:
filtered_imgs,good_img_connectivity = make_mosaic(x_test,return_connectivity=True,plot_images=True);

In [ ]:
filtered_imgs.shape

In [ ]:
test

In [ ]:
test.iloc[175,:]

In [ ]:
test.iloc[1523,:]

In [ ]:
test.iloc[12757,:]

In [ ]:
test.iloc[5335,:].z

In [ ]:
for el in [175, 1523, 12757, 5335]:
    print(test.iloc[el,:].z)

In [ ]:
for el in [7569, 8689, 5412, 246]:
    print(test.iloc[el,:].z)

In [ ]:
good_img_connectivity

In [ ]:
make_mosaic(x_train,return_connectivity=False,plot_images=True);

In [ ]:
good_img_connectivity

In [ ]:
Визуально картинки одной глубины, бывают сильно похожими на соседние, но крайне редко.
d = 655
width = 5
height = len(depths[depths.z == d].index.values)//width + (len(depths[depths.z == d].index.values)%width >0)
fig, axs = plt.subplots(height, width, figsize=(15, 14))
for i, names in enumerate(depths[depths.z == d].index.values):
   ax = axs[i // width, i % width]
   try:
       ax.imshow(np.asarray(Image.open(osp.join(path_train, 'images', f'{names}.png'))))
   except:
       ax.imshow(np.asarray(Image.open(osp.join(path_test, 'images', f'{names}.png'))))

In [ ]:
img = read_image(), self.input_shape)
mask = read_mask(os.path.join(args.masks_dir,'{}.png'.format('3577258d6b')), self.input_shape)